<a href="https://colab.research.google.com/github/Smart-Lizard/Med_Image_Generation/blob/main/ChestMNIST_Data_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This colab notebook will create a folder of ChestMNIST data where there is an equal amount of images for each of the labels.
#(the number can be chosen.)
#Additionally, a metadata file that works with the LoRA script will also be created.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

!pip install medmnist

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 972.3 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117030 sha256=57f669075e7b3a0c8c48ba63eea073bf592058b4ae90d68b83360c9c5bc42977
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire


In [ ]:
import medmnist
from medmnist import INFO, Evaluator
data_flag = 'chestmnist'
download = True

BATCH_SIZE = 128
info = INFO[data_flag]
DataClass = getattr(medmnist, info['python_class'])

# preprocessing
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

# load the data
train_dataset = DataClass(split='train', transform=data_transform, download=download, size=224, mmap_mode='r')

# encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)

100%|██████████| 3889293042/3889293042 [02:41<00:00, 24069456.09it/s]


In [ ]:
import os # Import the os module

chest = np.load("/root/.medmnist/chestmnist_224.npz")
train_images = chest['train_images']
train_labels = chest['train_labels']

In [ ]:
import numpy as np
import os
import json
from PIL import Image

# Find images with only one label
single_label_images = train_images[np.sum(train_labels, axis=1) == 1]

disease_names = [
    'Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax',
    'Consolidation', 'Edema', 'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia']

output_dir = '/content/drive/MyDrive/ChestMNIST_Data'
images_dir = os.path.join(output_dir, 'images_single_label')
os.makedirs(images_dir, exist_ok=True)

# Define the path to the metadata.jsonl file
metadata_path = os.path.join(images_dir, 'metadata.jsonl')

label_counts = {label: 0 for label in range(len(disease_names))}
# Open the metadata.jsonl file in write mode
with open(metadata_path, 'w') as f:
  for i in range(len(single_label_images)):
      image = single_label_images[i]
      label = np.argmax(train_labels[i])

      if label_counts[label] < 1000:

          image_pil = Image.fromarray(image)
          image_filename = f"image_{i}.png"
          image_path = os.path.join(images_dir, image_filename)
          image_pil.save(image_path)

          label_counts[label] += 1

          # Create a dictionary to store the metadata for the image
          metadata = {
            'file_name': image_filename,
            'label': disease_names[label]
          }

          # Write the metadata to the metadata.jsonl file
          f.write(json.dumps(metadata) + '\n')